In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

In [20]:
# Veri yükleme
data = pd.read_excel("C:/Users/aziz-/Downloads/aselsan.xlsx")

In [21]:
# Kapanış fiyatlarını ölçeklendirme
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Kapanış(TL)']].values)

In [22]:
# Create dataset function
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

In [23]:
# Data augmentation function
def augment_data(data, noise_level=0.01):
    noise = np.random.normal(0, noise_level, data.shape)
    augmented_data = data + noise
    return augmented_data

In [24]:
# TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)

In [ ]:
time_steps = [50, 75, 80]

for time_step in time_steps:
    # Create dataset
    X, Y = create_dataset(scaled_data, time_step)
    
    # Split data into training and testing sets
    train_size = int(len(X) * 0.8)
    test_size = len(X) - train_size
    X_train, X_test = X[:train_size], X[train_size:]
    Y_train, Y_test = Y[:train_size], Y[train_size:]
    
    # Reshape data for LSTM input
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    
    # Build model
    regression = Sequential()
    regression.add(LSTM(units=50, return_sequences=True, kernel_initializer='glorot_uniform', input_shape=(time_step, 1)))
    regression.add(Dropout(0.2))
    regression.add(LSTM(units=50, return_sequences=True, kernel_initializer='glorot_uniform'))
    regression.add(Dropout(0.1))
    regression.add(LSTM(units=50, kernel_initializer='glorot_uniform'))
    regression.add(Dropout(0.2))
    regression.add(Dense(units=1))
    
    regression.compile(optimizer='adam', loss='mean_squared_error')
    
    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
    
    # Model checkpoint
    model_path = f"regression_model_time_step_{time_step}.h5"
    model_checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
    
    # Fit model
    regression.fit(X_train, Y_train, epochs=600, batch_size=20, validation_split=0.1, callbacks=[early_stopping, model_checkpoint])
    
    # Evaluate model
    model_hata = regression.evaluate(X_test, Y_test, verbose=1)
    print(f'Time Step: {time_step}, Model Hata (MSE):', model_hata)
    
    # Make predictions
    train_predict = regression.predict(X_train)
    test_predict = regression.predict(X_test)
    
    # Inverse transform predictions
    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)
    Y_train = scaler.inverse_transform(Y_train.reshape(-1, 1))
    Y_test = scaler.inverse_transform(Y_test.reshape(-1, 1))
    
    # Performance metrics
    train_mae = mean_absolute_error(Y_train, train_predict)
    test_mae = mean_absolute_error(Y_test, test_predict)
    train_mse = mean_squared_error(Y_train, train_predict)
    test_mse = mean_squared_error(Y_test, test_predict)
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    train_r2 = r2_score(Y_train, train_predict)
    test_r2 = r2_score(Y_test, test_predict)
    
    # Plot results
    plt.figure(figsize=(14, 7))
    plt.plot(scaler.inverse_transform(scaled_data), color='blue', label='Gerçek Değerler (Tüm Veri)')
    train_indices = np.arange(time_step, time_step + len(train_predict))
    test_indices = np.arange(time_step + len(train_predict) + 1, time_step + len(train_predict) + 1 + len(test_predict))
    plt.plot(train_indices, train_predict, color='orange', label='Eğitim Tahminleri')
    plt.plot(test_indices, test_predict, color='red', label='Test Tahminleri')
    plt.legend()
    plt.show()
    
    # Print performance metrics
    print(f"Time Step: {time_step}, Train MAE: {train_mae}, Test MAE: {test_mae}")
    print(f"Time Step: {time_step}, Train MSE: {train_mse}, Test MSE: {test_mse}")
    print(f"Time Step: {time_step}, Train RMSE: {train_rmse}, Test RMSE: {test_rmse}")
    print(f"Time Step: {time_step}, Train R2: {train_r2}, Test R2: {test_r2}")

Epoch 1/600
17/17 [==============================] - ETA: 0s - loss: 0.0295
Epoch 1: val_loss improved from inf to 0.05768, saving model to regression_model_time_step_50.h5
17/17 [==============================] - 15s 192ms/step - loss: 0.0295 - val_loss: 0.0577
Epoch 2/600
 3/17 [====>.........................] - ETA: 0s - loss: 0.0108

C:\Users\aziz-\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


16/17 [===========================>..] - ETA: 0s - loss: 0.0077
Epoch 2: val_loss improved from 0.05768 to 0.02689, saving model to regression_model_time_step_50.h5
17/17 [==============================] - 1s 49ms/step - loss: 0.0077 - val_loss: 0.0269
Epoch 3/600
17/17 [==============================] - ETA: 0s - loss: 0.0042
Epoch 3: val_loss improved from 0.02689 to 0.00763, saving model to regression_model_time_step_50.h5
17/17 [==============================] - 1s 57ms/step - loss: 0.0042 - val_loss: 0.0076
Epoch 4/600
17/17 [==============================] - ETA: 0s - loss: 0.0031
Epoch 4: val_loss improved from 0.00763 to 0.00258, saving model to regression_model_time_step_50.h5
17/17 [==============================] - 1s 58ms/step - loss: 0.0031 - val_loss: 0.0026
Epoch 5/600
17/17 [==============================] - ETA: 0s - loss: 0.0030
Epoch 5: val_loss did not improve from 0.00258
17/17 [==============================] - 1s 56ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 